In [1]:
import torch
import tiktoken

In [2]:
with open ("Amontillado.txt", "r") as f:
    raw_text = f.read()

raw_text[:50]

'The thousand injuries of Fortunato I had borne as '

In [3]:
tokenizer = tiktoken.get_encoding("gpt2")


In [4]:
enc_text = tokenizer.encode(raw_text)

In [5]:
print(enc_text[:20])

[464, 7319, 6821, 286, 376, 1922, 5549, 314, 550, 28068, 355, 314, 1266, 714, 26, 475, 618, 339, 44716, 2402]


In [6]:
print(tokenizer.decode(enc_text[:20]))

The thousand injuries of Fortunato I had borne as I best could; but when he ventured upon


In [7]:
for i in range(1,10):
    print("INPUT:", tokenizer.decode(enc_text[:i]), "TARGET:", tokenizer.decode([enc_text[i]]))

INPUT: The TARGET:  thousand
INPUT: The thousand TARGET:  injuries
INPUT: The thousand injuries TARGET:  of
INPUT: The thousand injuries of TARGET:  F
INPUT: The thousand injuries of F TARGET: ortun
INPUT: The thousand injuries of Fortun TARGET: ato
INPUT: The thousand injuries of Fortunato TARGET:  I
INPUT: The thousand injuries of Fortunato I TARGET:  had
INPUT: The thousand injuries of Fortunato I had TARGET:  borne


## Data Set 

In [8]:
from torch.utils.data import Dataset, DataLoader


class GPTDatasetV1(Dataset):
    def __init__(self, txt, tokenizer, max_length, stride):
        self.input_ids = []
        self.target_ids = []

        # Tokenize the entire text
        token_ids = tokenizer.encode(txt, allowed_special={"<|endoftext|>"})
        assert len(token_ids) > max_length, "Number of tokenized inputs must at least be equal to max_length+1"

        # Use a sliding window to chunk the book into overlapping sequences of max_length
        for i in range(0, len(token_ids) - max_length, stride):
            input_chunk = token_ids[i:i + max_length]
            target_chunk = token_ids[i + 1: i + max_length + 1]
            self.input_ids.append(torch.tensor(input_chunk))
            self.target_ids.append(torch.tensor(target_chunk))

    def __len__(self):
        return len(self.input_ids)

    def __getitem__(self, idx):
        return self.input_ids[idx], self.target_ids[idx]



## Dataloader

In [9]:
def create_dataloader_v1(txt, batch_size=4, max_length=256, 
                         stride=128, shuffle=True, drop_last=True,
                         num_workers=0):

    # Initialize the tokenizer
    tokenizer = tiktoken.get_encoding("gpt2")

    # Create dataset
    dataset = GPTDatasetV1(txt, tokenizer, max_length, stride)

    # Create dataloader
    dataloader = DataLoader(
        dataset,
        batch_size=batch_size,
        shuffle=shuffle,
        drop_last=drop_last,
        num_workers=num_workers
    )

    return dataloader

In [10]:
dataloader = create_dataloader_v1(raw_text, batch_size=8, max_length=4, stride=4, shuffle=False)

data_iter = iter(dataloader)
inputs, targets = next(data_iter)
print("Inputs:\n", inputs)
print("\nTargets:\n", targets)


Inputs:
 tensor([[  464,  7319,  6821,   286],
        [  376,  1922,  5549,   314],
        [  550, 28068,   355,   314],
        [ 1266,   714,    26,   475],
        [  618,   339, 44716,  2402],
        [13277,    11,   314, 19982],
        [15827,    13,   921,    11],
        [  508,   523,   880,   760]])

Targets:
 tensor([[ 7319,  6821,   286,   376],
        [ 1922,  5549,   314,   550],
        [28068,   355,   314,  1266],
        [  714,    26,   475,   618],
        [  339, 44716,  2402, 13277],
        [   11,   314, 19982, 15827],
        [   13,   921,    11,   508],
        [  523,   880,   760,   262]])


In [11]:
type(inputs)

torch.Tensor

In [12]:
#Gives dimensions of the tensor
inputs.shape

torch.Size([8, 4])

In [13]:
#Look at the 2nd row
inputs[1]

tensor([ 376, 1922, 5549,  314])

In [14]:
# Look at the first column, first element of each row
inputs[:,0]

tensor([  464,   376,   550,  1266,   618, 13277, 15827,   508])

In [15]:
# Look at the ID in the 2nd row and 3rd column
inputs[1,2]

tensor(5549)

In [16]:
#Print tensor row by row, converting to list so tokenizer can decode
for row in inputs:
    print(tokenizer.decode(row.tolist()))

The thousand injuries of
 Fortunato I
 had borne as I
 best could; but
 when he ventured upon
 insult, I vowed
 revenge. You,
 who so well know


## Example of Generating Embedding Vectors

In [17]:
#illustrative example of how to generate the vectors of length 3 for the 6 word vocabular
vocab_size = 6
output_dim = 3

In [18]:
#nn is a sub-module for nueral networks in torch which contains Embedding
#Embedding created embedding vectors of a certain length for a certain vocab size
embedding = torch.nn.Embedding(vocab_size, output_dim)

In [19]:
print(embedding.weight)

Parameter containing:
tensor([[ 2.1105, -0.3783,  0.5331],
        [-0.0432,  0.2590,  0.3422],
        [ 0.7260,  0.2596, -1.8951],
        [ 0.1433,  0.2822, -1.3152],
        [ 0.9741, -0.5730,  2.7245],
        [ 1.0162, -1.2116,  0.3251]], requires_grad=True)


In [20]:
embedding.weight[0]

tensor([ 2.1105, -0.3783,  0.5331], grad_fn=<SelectBackward0>)

In [21]:
#get rid of reqires_grad bit of text
embedding.weight.data

tensor([[ 2.1105, -0.3783,  0.5331],
        [-0.0432,  0.2590,  0.3422],
        [ 0.7260,  0.2596, -1.8951],
        [ 0.1433,  0.2822, -1.3152],
        [ 0.9741, -0.5730,  2.7245],
        [ 1.0162, -1.2116,  0.3251]])

In [22]:
# Or get rid of it using detach()
print(embedding.weight.detach())

tensor([[ 2.1105, -0.3783,  0.5331],
        [-0.0432,  0.2590,  0.3422],
        [ 0.7260,  0.2596, -1.8951],
        [ 0.1433,  0.2822, -1.3152],
        [ 0.9741, -0.5730,  2.7245],
        [ 1.0162, -1.2116,  0.3251]])


In [23]:
#What it would really look like with the correct sizes
#embedding = torch.nn.Embedding(tokenizer.n_vocab, 256)

In [24]:
A = embedding.weight.data
A

tensor([[ 2.1105, -0.3783,  0.5331],
        [-0.0432,  0.2590,  0.3422],
        [ 0.7260,  0.2596, -1.8951],
        [ 0.1433,  0.2822, -1.3152],
        [ 0.9741, -0.5730,  2.7245],
        [ 1.0162, -1.2116,  0.3251]])

In [25]:
#making tensors from scratch-- just put in the elements
x = torch.tensor([2.1,1.3])
y = torch.tensor([1.5,2.7])
print(x,y)

tensor([2.1000, 1.3000]) tensor([1.5000, 2.7000])


In [26]:
#computing the dot product if two tensors
torch.dot(x,y)

tensor(6.6600)

## Some More Practice Stuff on Attention Mechanisms

In [27]:
raw_text= "This is my dog Hazel"

In [58]:
tokenizer= tiktoken.get_encoding("gpt2")
enc_text = tokenizer.encode(raw_text)
print (enc_text)

[1212, 318, 616, 3290, 42805]


In [59]:
vocab_size = 4
output_dim = 8
inputs = torch.nn.Embedding(vocab_size, output_dim)
print(inputs.weight)

Parameter containing:
tensor([[-1.2155,  1.9469, -0.4953, -2.5550, -0.0844,  0.2847,  1.5802,  0.9100],
        [ 0.6381,  1.3723,  0.5784,  1.1823, -0.5780,  2.5947,  1.6177,  0.3148],
        [-0.7139, -1.5165,  1.2933,  0.9406,  0.0263,  0.7339, -1.1206, -0.9787],
        [-0.0267,  0.2666, -0.5321, -0.8025, -2.3368,  1.1187,  1.6303,  0.5704]],
       requires_grad=True)


In [60]:
#This gets rid of the grad = True thing
inputs = inputs.weight.data
inputs

tensor([[-1.2155,  1.9469, -0.4953, -2.5550, -0.0844,  0.2847,  1.5802,  0.9100],
        [ 0.6381,  1.3723,  0.5784,  1.1823, -0.5780,  2.5947,  1.6177,  0.3148],
        [-0.7139, -1.5165,  1.2933,  0.9406,  0.0263,  0.7339, -1.1206, -0.9787],
        [-0.0267,  0.2666, -0.5321, -0.8025, -2.3368,  1.1187,  1.6303,  0.5704]])

In [61]:
inputs.shape

torch.Size([4, 8])

In [62]:
for row in inputs:
    print(row.tolist())

[-1.2155163288116455, 1.9468598365783691, -0.4952908158302307, -2.5550310611724854, -0.08435238897800446, 0.28469333052635193, 1.580206036567688, 0.9099704623222351]
[0.6380577087402344, 1.3723042011260986, 0.5784104466438293, 1.1822501420974731, -0.5779953598976135, 2.5946590900421143, 1.6177409887313843, 0.31481605768203735]
[-0.713903546333313, -1.5164538621902466, 1.2932689189910889, 0.9406096339225769, 0.02626447193324566, 0.7339489459991455, -1.1205947399139404, -0.9786593914031982]
[-0.026710473001003265, 0.26656144857406616, -0.5321037173271179, -0.8024697303771973, -2.3367929458618164, 1.1186763048171997, 1.630319595336914, 0.5703837275505066]


In [63]:
#Example to do dot product
x = torch.Tensor([1.1, 2.3])
y = torch.Tensor([3.4,-2.1])

In [64]:
#Calculate the dot product... annoying by hand
1.1*3.4 + 2.3*(-2.1)

-1.0899999999999999

In [65]:
#Built in dot product method!
torch.dot(x,y)

tensor(-1.0900)

In [66]:
#The important one is called the "query"
query = inputs[2]
print(query)

tensor([-0.7139, -1.5165,  1.2933,  0.9406,  0.0263,  0.7339, -1.1206, -0.9787])


In [67]:
# gives the dot products of the 3rd embedding vector with all the other vectors (vector 3 x vector 1, vector 3 x vector 2 ...)
for i in range(len(inputs)):
    print(torch.dot(query, inputs[i]))

tensor(-7.5830)
tensor(-0.9082)
tensor(8.1195)
tensor(-3.4536)


In [68]:
#Want the dot products (attention scores) in a tensor of their own
#torch.zeros creates a tensor of zeros so that you can fill them in

attention_scores_2 = torch.zeros(len(inputs))
for i in range(len(inputs)):
    attention_scores_2[i] = (torch.dot(query, inputs[i]))
print(attention_scores_2)

tensor([-7.5830, -0.9082,  8.1195, -3.4536])


In [69]:
# Normaling the attention scores using softmax function-- basically makes everything positive in a complicated way
#def softmax(x):
    #torch.exp(x, dim=0) / torch.exp(x).sum()

In [70]:
#use the real torch function!
# Make sure you add the dimension
attention_weights_2 = torch.softmax(attention_scores_2, dim = 0)
attention_weights_2

tensor([1.5152e-07, 1.2002e-04, 9.9987e-01, 9.4153e-06])

In [71]:
attention_weights_2.sum()

tensor(1.)

In [72]:
# Making the actual context vector ...
context_vector_2 = torch.zeros(query.shape)
for i in range(len(attention_weights_2)):
    context_vector_2 += attention_weights_2[i]*inputs[i]
context_vector_2

tensor([-0.7137, -1.5161,  1.2932,  0.9406,  0.0262,  0.7342, -1.1202, -0.9785])

In [73]:
#Get all attention scores for all input vectors via matrix multiplication (@) easier than by for loops
attention_scores = inputs @ inputs.T
attention_scores

tensor([[15.4545,  2.2192, -7.5830,  6.4762],
        [ 2.2192, 13.8051, -0.9082,  6.1625],
        [-7.5830, -0.9082,  8.1195, -3.4536],
        [ 6.4762,  6.1625, -3.4536, 10.6942]])

In [74]:
#Normalize them 
attention_weights = torch.softmax(attention_scores, dim = -1)
attention_weights

tensor([[9.9987e-01, 1.7862e-06, 9.8831e-11, 1.2610e-04],
        [9.2915e-06, 9.9951e-01, 4.0724e-07, 4.7933e-04],
        [1.5152e-07, 1.2002e-04, 9.9987e-01, 9.4153e-06],
        [1.4362e-02, 1.0495e-02, 6.9948e-07, 9.7514e-01]])

In [75]:
attention_weights[0].sum()

tensor(1.)

In [76]:
context_vectors = attention_weights @ inputs
context_vectors

tensor([[-1.2154,  1.9466, -0.4953, -2.5548, -0.0846,  0.2848,  1.5802,  0.9099],
        [ 0.6377,  1.3718,  0.5779,  1.1813, -0.5788,  2.5939,  1.6177,  0.3149],
        [-0.7137, -1.5161,  1.2932,  0.9406,  0.0262,  0.7342, -1.1202, -0.9785],
        [-0.0368,  0.3023, -0.5199, -0.8068, -2.2860,  1.1222,  1.6295,  0.5726]])